In [186]:
import pandas as pd # Make sure that you have openpyxl installed so that you can open newer excel files
import numpy as np
import os 
from zipfile import is_zipfile, ZipFile
from datetime import datetime, date
from plotnine import ggplot, geom_line, aes, theme_minimal
from plotnine3d import ggplot_3d, geom_polygon_3d
import matplotlib.pyplot as plt

In [187]:
def extractFolder(inputFilesDir,unzippedFilesDir):
    for file in os.listdir(inputFilesDir):   # get the list of files
        if is_zipfile(inputFilesDir+file): # if it is a zipfile, extract it
            with ZipFile(inputFilesDir+file) as item: # treat the file as a zip
                item.extractall(unzippedFilesDir)  # extract it into a new folder

def cleanTermStructureSheet(filename,foldername,sheetName):
    df = pd.read_excel(foldername+filename, sheet_name=sheetName, header=1, engine='openpyxl')
    df.drop(['Unnamed: 0'], axis=1, inplace=True) # Blank column - artifact of the dataset
    df = df.iloc[8:,:] # Dropping all unnessary columns 
    df.rename(columns={"Main menu": "Tenor"}, inplace=True)
    df.set_index("Tenor", inplace=True)
    return df

def getDateFromFileName(filename):
    dateString = filename.split('_')[2]
    yearString = dateString[:4]
    monthString = dateString[4:6]
    dayString = dateString[6:]
    return date(int(yearString),int(monthString),int(dayString))

def getCountrieslist(inputFilesDir):
    return pd.read_csv(inputFilesDir+"countries.csv").columns.to_list()

def getFilteredFileList(unzippedFilesDir,fileType,sheetName):
    allFiles = os.listdir(unzippedFilesDir)
    filteredList = [file for file in allFiles if fileType in file]
    return filteredList 

inputFilesDir = "inputData/"
unzippedFilesDir = "unzippedFiles/"
fileType = "Term_Structures"
sheetName = "RFR_spot_no_VA"

extractFolder(inputFilesDir,unzippedFilesDir)
countries = getCountrieslist(inputFilesDir)

countriesDict = {}
for country in countries:
    countriesDict[country] = {}

for filename in getFilteredFileList(unzippedFilesDir,fileType,sheetName):
    dateIndex = getDateFromFileName(filename)
    allTermStructures = cleanTermStructureSheet(filename,unzippedFilesDir,sheetName)
    for country in countries:
        countriesDict[country][dateIndex] =  allTermStructures.loc[:,country]


In [188]:
usdData = pd.DataFrame(countriesDict['South Africa']).sort_index(axis = 1)
dates = usdData.columns
tenorStructure = usdData.index.values
usdData

,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29
Tenor,,,,,,,,,,,,,,,
1,0.07756,0.07606,0.07978,0.08251,0.08539,0.09179,0.08805,0.08592,0.08368,0.087,0.08528,0.08313,0.08239,0.07964,0.08093
2,0.07834,0.07376,0.08012,0.07998,0.08384,0.09091,0.08628,0.08359,0.0807,0.08622,0.08363,0.07895,0.07866,0.07569,0.07884
3,0.07982,0.07404,0.08104,0.07969,0.08353,0.09222,0.08608,0.08341,0.0803,0.08653,0.08395,0.07835,0.07873,0.07531,0.07911
4,0.08159,0.07589,0.08258,0.08091,0.08452,0.09386,0.0872,0.08428,0.0816,0.08843,0.08603,0.07995,0.07995,0.07677,0.0808
5,0.08418,0.07846,0.08507,0.08304,0.08667,0.09662,0.08923,0.08618,0.08436,0.09147,0.08911,0.08232,0.08208,0.0795,0.08348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,0.06193,0.06139,0.0626,0.06238,0.06304,0.06645,0.0636,0.06324,0.06321,0.06419,0.06399,0.06319,0.0635,0.06189,0.06268
147,0.06188,0.06135,0.06255,0.06233,0.06298,0.06637,0.06354,0.06318,0.06315,0.06412,0.06393,0.06313,0.06345,0.06183,0.06262
148,0.06184,0.06131,0.0625,0.06228,0.06293,0.06629,0.06348,0.06313,0.0631,0.06406,0.06386,0.06307,0.06339,0.06177,0.06256


In [205]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
# Read data from a csv

fig = go.Figure(data=[go.Surface(z=usdData.T.values, 
                      x=tenorStructure, 
                      y=dates,
                      opacity = 0.8
                      
                      )])
fig.update_layout(scene=dict(xaxis_title='Tenor',
                           yaxis_title='Date',
                           zaxis_title='Yield',
                           aspectratio = dict(x = 0.8, y = 1.4, z= 0.75)),
                  title='United States Curve over Time', autosize=False,
                  width=1000, height=800,
                  margin=dict(l=50, r=20, b=100, t=50), 
                  paper_bgcolor="LightSteelBlue"
                  )

fig.show()
# {"scene": {"aspectratio": {"x": 0.8, "y": 1.4, "z": 0.75}}}
# fig.update_layout({"scene": {"aspectratio": {"x": 2, "y": 2, "z": 0.75}}})

In [191]:
tenorStructure.values

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [ ]:
tenorStructure[-1:0]

Index([], dtype='object', name='Tenor')